In [ ]:
# basics
import pandas as pd
import numpy as np

# mapltplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# data management
import os
import time
import copy

# Create a time stamp for this run
tstamp = time.strftime("%d_%m_%Y_%H_%M", time.localtime())

CWD = "/".join(os.getcwd().split("/")[:-2])

In [ ]:
inits = pd.read_csv(f"{CWD}/data/summary/inits_decoupled_GP.csv")#_decoupled    
target = inits.iloc[5]
target.tstamp, target.ID

In [ ]:
# get TESS ID without suffix
ID = target.ID.strip("ab")
# get light curve from GP fitting
lc = pd.read_csv(f"{CWD}/data/lcs/{target.tstamp}_{ID}.csv")
# make a copy
cff = copy.deepcopy(lc)


In [ ]:
ID

In [ ]:
lc.shape

In [ ]:
# get median
median = lc.median_[0]

# new figure
plt.figure(figsize=(12,6))

# plot the light curve as is
plt.errorbar(cff.phi / (2 * np.pi), cff.flux, yerr=cff.flux_err, 
             color="grey", alpha=.1, errorevery=1,zorder=-3)

# define A/2
half = cff.flux > cff.flux.max() / 2 + median / 2

# # if there is a second flare, capture it separately
# second = .5 # approximately before the onset of the second flare
# cond2 =  (cff.phi / (2 * np.pi) > second) 
# half2 = (cff.flux - median) > (cff.loc[cond2, "flux"].max()*.5 - median/2) 

# increase the uncertainty to 20% of the flare-only flux above A/2 for FIRST flare
cff.loc[half,  "flux_err" ] = (np.max(cff[half].flux)  - median) * .2

# # increase the uncertainty to 20% of the flare-only flux above A/2 for SECOND flare
# cff.loc[half2 & cond2,  "flux_err" ] = (np.max(cff[half2&cond2].flux)  - median) * .2

# plot new uncertainties
plt.errorbar(cff.phi/(2*np.pi), cff.flux, yerr=cff.flux_err, color="r", alpha=.3, errorevery=1)

# plot median
plt.plot(cff.phi/(2*np.pi), cff.median_, color="k", zorder=1)

# layout
plt.xlabel("time [rotation period]")
plt.ylabel(r"flux [e$^{-}$/s]")
tpeak = target.phi_a/(2*np.pi)
plt.xlim(tpeak-.65,tpeak+1.8);

In [ ]:
tstamp, ID

In [ ]:
# save LC with increased uncertainties to file
cff.to_csv(f"{CWD}/data/lcs/{tstamp}_{ID}.csv")